# Interpolation
- Already have a tool in mu2e.tools.mapinterp (a factory function to get an interpolator for a given map file).
- Uses scipy.interpolate griddata function (can do nearest or linear interp).
- Gathers 8 nearest points in a stupid way...query is pretty slow.
- Note that trajectory code currently uses actual model evaluation...I'm not sure that any practical applications do it this way (check!). Seems that everyone just interpolates a map!
    - This calculation is definitely the bottleneck so a quick interpolation might fix things.
    - (Though eventually will want a quick model evaluation for comparing good vs. bad)

- The goal of this notebook is to work through details of index based data selection to quickly pull the 8 nearest points, then to do a linear or trilinear interpolation. It will be a useful tool in general, but may be very useful for getting the trajectory code running.

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly import tools
init_notebook_mode(True)

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo

from mu2e import mu2e_ext_path
from mu2e.dataframeprod import DataFrameMaker
from mu2e.mu2eplots import mu2e_plot, mu2e_plot3d

In [6]:
from hallprobecalib.hallrawdataframe import gradient_calc

In [5]:
plotdir = mu2e_ext_path + 'plots/interp_test/'

In [7]:
df = DataFrameMaker(mu2e_ext_path+"datafiles/Mau13/Mu2e_DSMap_V13",input_type='pkl').data_frame

## Current Interpolation Factory Function

In [21]:
from mu2e.tools.mapinterp import get_ds_interp_func

In [22]:
mau13_nearest = get_ds_interp_func(df,method='nearest')
mau13_interp = get_ds_interp_func(df,method='linear')

## Index Based Selection

In [13]:
%timeit mau13_interp(0.156,-.699,7.001)

38.6 ms ± 261 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit mau13_nearest(0.156,-.699,7.001)

38.5 ms ± 615 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
mau13_interp(0.156,-.699,7.001)

(94.57883452204001, -446.1719677802401, 11347.763332736002)

In [24]:
mau13_nearest(0.156,-.699,7.001)

(96.407603304, -472.752239114, 11351.8362832)

- Both nearest and linear are about the same speed, but I think most of this time is spent selecting the data.

- Let's also see the current speed of model evaluation.

In [106]:
from mu2e.tools.modeleval import get_mag_field_function

In [108]:
mag_field_func = get_mag_field_function('Mau13traj',units=('m','G'),fastcart=True)

recreating fit with func_version=1000,
cfg_params(pitch1=0, ms_h1=0, ns_h1=0, pitch2=0, ms_h2=0, ns_h2=0, length1=10, ms_c1=50, ns_c1=4, length2=0, ms_c2=0, ns_c2=0, ks_dict={'k3': 10000}, bs_tuples=((1.0, 0, 7.873), (1.0, 0, 13.389)), bs_bounds=(0.1, 0.1, 5), version=1000)
Elapsed time was 1.83963 seconds
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 0
    # data points      = 46560
    # variables        = 0
    chi-square         = 764307.395
    reduced chi-square = 16.4155368
    Akaike info crit   = 130285.508
    Bayesian info crit = 130285.508
[[Variables]]
    pitch1:    0 (fixed)
    ms_h1:     0 (fixed)
    ns_h1:     0 (fixed)
    pitch2:    0 (fixed)
    ms_h2:     0 (fixed)
    ns_h2:     0 (fixed)
    length1:   10 (fixed)
    ms_c1:     50 (fixed)
    ns_c1:     4 (fixed)
    length2:   0 (fixed)
    ms_c2:     0 (fixed)
    ns_c2:     0 (fixed)
    Ac1_0_0:  -74613.8 (fixed)
    Bc1_0_0:   13334 (fixed)
    Dc1_0:     0.9452835 (fixed)
  

In [109]:
mag_field_func(0.156,-.699,7.001)

(4.929430190673969, -21.534389481737904, 12223.661351884144)

In [110]:
%timeit mag_field_func(0.156,-.699,7.001)

2.2 ms ± 34.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
df.X.unique()

array([-1.2  , -1.175, -1.15 , -1.125, -1.1  , -1.075, -1.05 , -1.025,
       -1.   , -0.975, -0.95 , -0.925, -0.9  , -0.875, -0.85 , -0.825,
       -0.8  , -0.775, -0.75 , -0.725, -0.7  , -0.675, -0.65 , -0.625,
       -0.6  , -0.575, -0.55 , -0.525, -0.5  , -0.475, -0.45 , -0.425,
       -0.4  , -0.375, -0.35 , -0.325, -0.3  , -0.275, -0.25 , -0.225,
       -0.2  , -0.175, -0.15 , -0.125, -0.1  , -0.075, -0.05 , -0.025,
        0.   ,  0.025,  0.05 ,  0.075,  0.1  ,  0.125,  0.15 ,  0.175,
        0.2  ,  0.225,  0.25 ,  0.275,  0.3  ,  0.325,  0.35 ,  0.375,
        0.4  ,  0.425,  0.45 ,  0.475,  0.5  ,  0.525,  0.55 ,  0.575,
        0.6  ,  0.625,  0.65 ,  0.675,  0.7  ,  0.725,  0.75 ,  0.775,
        0.8  ,  0.825,  0.85 ,  0.875,  0.9  ,  0.925,  0.95 ,  0.975,
        1.   ,  1.025,  1.05 ,  1.075,  1.1  ,  1.125,  1.15 ,  1.175,
        1.2  ])

In [17]:
df.Y.unique()

array([-1.2  , -1.175, -1.15 , -1.125, -1.1  , -1.075, -1.05 , -1.025,
       -1.   , -0.975, -0.95 , -0.925, -0.9  , -0.875, -0.85 , -0.825,
       -0.8  , -0.775, -0.75 , -0.725, -0.7  , -0.675, -0.65 , -0.625,
       -0.6  , -0.575, -0.55 , -0.525, -0.5  , -0.475, -0.45 , -0.425,
       -0.4  , -0.375, -0.35 , -0.325, -0.3  , -0.275, -0.25 , -0.225,
       -0.2  , -0.175, -0.15 , -0.125, -0.1  , -0.075, -0.05 , -0.025,
        0.   ,  0.025,  0.05 ,  0.075,  0.1  ,  0.125,  0.15 ,  0.175,
        0.2  ,  0.225,  0.25 ,  0.275,  0.3  ,  0.325,  0.35 ,  0.375,
        0.4  ,  0.425,  0.45 ,  0.475,  0.5  ,  0.525,  0.55 ,  0.575,
        0.6  ,  0.625,  0.65 ,  0.675,  0.7  ,  0.725,  0.75 ,  0.775,
        0.8  ,  0.825,  0.85 ,  0.875,  0.9  ,  0.925,  0.95 ,  0.975,
        1.   ,  1.025,  1.05 ,  1.075,  1.1  ,  1.125,  1.15 ,  1.175,
        1.2  ])

In [34]:
df.Z.unique()

array([ 3.071,  3.096,  3.121,  3.146,  3.171,  3.196,  3.221,  3.246,
        3.271,  3.296,  3.321,  3.346,  3.371,  3.396,  3.421,  3.446,
        3.471,  3.496,  3.521,  3.546,  3.571,  3.596,  3.621,  3.646,
        3.671,  3.696,  3.721,  3.746,  3.771,  3.796,  3.821,  3.846,
        3.871,  3.896,  3.921,  3.946,  3.971,  3.996,  4.021,  4.046,
        4.071,  4.096,  4.121,  4.146,  4.171,  4.196,  4.221,  4.246,
        4.271,  4.296,  4.321,  4.346,  4.371,  4.396,  4.421,  4.446,
        4.471,  4.496,  4.521,  4.546,  4.571,  4.596,  4.621,  4.646,
        4.671,  4.696,  4.721,  4.746,  4.771,  4.796,  4.821,  4.846,
        4.871,  4.896,  4.921,  4.946,  4.971,  4.996,  5.021,  5.046,
        5.071,  5.096,  5.121,  5.146,  5.171,  5.196,  5.221,  5.246,
        5.271,  5.296,  5.321,  5.346,  5.371,  5.396,  5.421,  5.446,
        5.471,  5.496,  5.521,  5.546,  5.571,  5.596,  5.621,  5.646,
        5.671,  5.696,  5.721,  5.746,  5.771,  5.796,  5.821,  5.846,
      

Index selection works as follow: Mau13 data is organized as first increasing in z, then increasing in y, then increasing in x. One can determine index by adding up steps to get to correct x value, then to y value, then to z value. See below. Makes sense to make a factory fcn as the selection will depend on the length of X,Y,Z unique.

In [33]:
xs = df.X.unique()
ys = df.Y.unique()
zs = df.Z.unique()

dx = xs[1]-xs[0]
dy = ys[1]-ys[0]
dz = zs[1]-zs[0]

lx = len(xs)
ly = len(ys)
lz = len(zs)

In [30]:
x_in, y_in, z_in = 0.156,-.699,7.001

In [32]:
n

False

In [28]:
lx

97

In [55]:
x_m, y_m, z_m = -1.1, 0.4, 7.096

In [56]:
ind_correct = df.query(f"X == {x_m} and Y == {y_m} and Z == {z_m}").index[0]

In [57]:
ind_correct

235653

In [58]:
x_ind = np.where(xs == x_m)[0][0]
y_ind = np.where(ys == y_m)[0][0]
z_ind = np.where(zs == z_m)[0][0]

In [59]:
x_ind, y_ind, z_ind

(4, 64, 161)

In [60]:
ind = (lz * ly) * x_ind + (lz) * y_ind + z_ind

In [61]:
ind

235653

Great, we got it right! Check a few more times--good!

Now we need to find 4 points (a square) immediately lower than the point, then the top of the square is just each index + 1. Really, if we can find the point with smallest x,y,z then all other 8 points can be calculated quickly.

In [ ]:
corner_a = 

In [51]:
lx,ly,lz

(97, 97, 521)

In [62]:
xs

array([-1.2  , -1.175, -1.15 , -1.125, -1.1  , -1.075, -1.05 , -1.025,
       -1.   , -0.975, -0.95 , -0.925, -0.9  , -0.875, -0.85 , -0.825,
       -0.8  , -0.775, -0.75 , -0.725, -0.7  , -0.675, -0.65 , -0.625,
       -0.6  , -0.575, -0.55 , -0.525, -0.5  , -0.475, -0.45 , -0.425,
       -0.4  , -0.375, -0.35 , -0.325, -0.3  , -0.275, -0.25 , -0.225,
       -0.2  , -0.175, -0.15 , -0.125, -0.1  , -0.075, -0.05 , -0.025,
        0.   ,  0.025,  0.05 ,  0.075,  0.1  ,  0.125,  0.15 ,  0.175,
        0.2  ,  0.225,  0.25 ,  0.275,  0.3  ,  0.325,  0.35 ,  0.375,
        0.4  ,  0.425,  0.45 ,  0.475,  0.5  ,  0.525,  0.55 ,  0.575,
        0.6  ,  0.625,  0.65 ,  0.675,  0.7  ,  0.725,  0.75 ,  0.775,
        0.8  ,  0.825,  0.85 ,  0.875,  0.9  ,  0.925,  0.95 ,  0.975,
        1.   ,  1.025,  1.05 ,  1.075,  1.1  ,  1.125,  1.15 ,  1.175,
        1.2  ])

In [66]:
x_in

0.156

In [68]:
%timeit xs[xs < x_in][-1]

1.23 µs ± 8.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [89]:
%timeit a_x, a_y, a_z = len(xs[xs <= x_in]) - 1, len(ys[ys <= y_in]) - 1, len(zs[zs <= z_in]) - 1

4.38 µs ± 84.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [87]:
a_x, a_y, a_z = len(xs[xs <= x_in]) - 1, len(ys[ys <= y_in]) - 1, len(zs[zs <= z_in]) - 1

In [88]:
a_x, a_y, a_z

(54, 20, 157)

In [74]:
%timeit corner_a = (ly * lz) * a_x + (lz) * a_y + a_z

656 ns ± 6.91 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [91]:
corner_a = (ly * lz) * a_x + (lz) * a_y + a_z

In [92]:
corner_b = corner_a + lz
corner_c = corner_a + ly * lz
corner_d = corner_a + ly * lz + lz

In [93]:
index_list = [corner_a,corner_a+1,corner_b,corner_b+1,
              corner_c,corner_c+1,corner_d,corner_d+1]

In [94]:
index_list

[2739575, 2739576, 2740096, 2740097, 2790112, 2790113, 2790633, 2790634]

In [98]:
df2 = df.drop(["X","Y","Z","R","Phi","Br","Bphi"],axis=1)

In [99]:
%timeit df_cube = df2.iloc[index_list]

141 µs ± 983 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [97]:
%timeit df_cube = df.iloc[index_list]

147 µs ± 6.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Once we have a list of indexes for the 8 points, we can feed them in to get a smaller dataframe as follows:

In [102]:
df_cube = df.iloc[index_list]

In [103]:
df_cube

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
2739575,0.150,-0.700,6.996,96.407603,-472.752239,11351.836283,0.715891,-1.359703,-4.787759,482.458478
2739576,0.150,-0.700,7.021,66.997243,-341.088893,11347.308917,0.715891,-1.359703,-5.957979,347.555415
2740096,0.150,-0.675,6.996,99.734601,-469.852881,11475.801077,0.691466,-1.352127,-4.565774,480.299775
2740097,0.150,-0.675,7.021,73.640047,-357.231036,11466.469056,0.691466,-1.352127,-5.607831,364.699084
2790112,0.175,-0.700,6.996,112.189233,-469.030206,11318.778439,0.721543,-1.325818,-4.916991,482.236023
2790113,0.175,-0.700,7.021,77.059626,-333.451222,11315.505317,0.721543,-1.325818,-6.114982,342.184907
2790633,0.175,-0.675,6.996,116.085160,-466.478112,11446.457588,0.697316,-1.317122,-4.698279,480.682347
2790634,0.175,-0.675,7.021,84.896775,-350.428549,11438.276190,0.697316,-1.317122,-5.764490,360.519627


This is just the format we need to run the griddata command, (and hopefully the trilinear interpolation too!).
Let's gather these steps into a function to see where we're at for timing.

In [111]:
xs = df.X.unique()
ys = df.Y.unique()
zs = df.Z.unique()

dx = xs[1]-xs[0]
dy = ys[1]-ys[0]
dz = zs[1]-zs[0]

lx = len(xs)
ly = len(ys)
lz = len(zs)

def get_cube(x,y,z):
    a_x, a_y, a_z = len(xs[xs <= x]) - 1, len(ys[ys <= y]) - 1, len(zs[zs <= z]) - 1
    corner_a = (ly * lz) * a_x + (lz) * a_y + a_z
    corner_b = corner_a + lz
    corner_c = corner_a + ly * lz
    corner_d = corner_a + ly * lz + lz
    index_list = [corner_a,corner_a+1,corner_b,corner_b+1,
                  corner_c,corner_c+1,corner_d,corner_d+1]
    return df.iloc[index_list]

In [115]:
get_cube(1,1,6)

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
4493221,1.000,1.000,5.996,803.117989,808.375168,-421.964785,1.414214,0.785398,3.717387,1139.497739
4493222,1.000,1.000,6.021,814.317279,819.308683,-354.092045,1.414214,0.785398,3.529455,1155.147996
4493742,1.000,1.025,5.996,752.897610,777.774796,-407.541777,1.432000,0.797743,4.228174,1082.483518
4493743,1.000,1.025,6.021,761.765729,786.623548,-348.430264,1.432000,0.797743,4.059828,1095.010114
4543758,1.025,1.000,5.996,771.438167,758.391541,-407.791397,1.432000,0.773053,4.129302,1081.784416
4543759,1.025,1.000,6.021,780.510716,767.012055,-348.713291,1.432000,0.773053,3.964133,1094.298294
4544279,1.025,1.025,5.996,726.232899,732.721952,-396.167319,1.449569,0.785398,4.588453,1031.636869
4544280,1.025,1.025,6.021,733.422256,739.699079,-344.411626,1.449569,0.785398,4.438384,1041.654085


In [114]:
%timeit get_cube(1,1,6)

161 µs ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [116]:
test_cube = get_cube(0.3,0.7,6.)

In [121]:
test_cube

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
3071933,0.300,0.700,5.996,606.750687,1387.768884,14689.461393,0.761577,1.165905,-11.022933,1514.571678
3071934,0.300,0.700,6.021,621.542007,1420.865136,14491.907756,0.761577,1.165905,-11.581049,1550.818520
3072454,0.300,0.725,5.996,644.054087,1524.945714,14741.128184,0.784618,1.178456,-12.051089,1655.330743
3072455,0.300,0.725,6.021,662.761117,1568.378528,14515.159227,0.784618,1.178456,-12.730086,1702.616060
3122470,0.325,0.700,5.996,673.386331,1424.128368,14711.884665,0.771767,1.136126,-11.050883,1575.267798
3122471,0.325,0.700,6.021,691.174252,1461.007069,14502.099405,0.771767,1.136126,-11.654649,1616.207806
3122991,0.325,0.725,5.996,716.389323,1568.364912,14768.911133,0.794512,1.149377,-12.163010,1724.190889
3122992,0.325,0.725,6.021,738.785185,1616.527557,14528.472062,0.794512,1.149377,-12.898228,1777.300911


In [118]:
from scipy.interpolate import griddata

In [119]:
bx = griddata((test_cube.X.values,test_cube.Y.values,test_cube.Z.values),test_cube.Bx.values, (0.3,0.7,6.), method='linear')

In [120]:
bx

array(609.11729831)

In [122]:
%timeit bx = griddata((test_cube.X.values,test_cube.Y.values,test_cube.Z.values),test_cube.Bx.values, (0.3,0.7,6.), method='linear')

266 µs ± 4.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [124]:
xvs = test_cube.X.values
yvs = test_cube.Y.values
zvs = test_cube.Z.values

In [127]:
bxs = test_cube.Bx.values

In [130]:
mag_field_func(0.3,0.7,6.)

(13.828397663404894, 31.93655140762153, 14532.908168464091)

In [129]:
%timeit bx = griddata((xvs,yvs,zvs),bxs, (0.3,0.7,6.), method='linear')

231 µs ± 3.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [126]:
%timeit bx = griddata((xvs,yvs,zvs),test_cube.Bx.values, (0.3,0.7,6.), method='linear')

249 µs ± 4.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [162]:
xs = df.X.unique()
ys = df.Y.unique()
zs = df.Z.unique()

dx = xs[1]-xs[0]
dy = ys[1]-ys[0]
dz = zs[1]-zs[0]

lx = len(xs)
ly = len(ys)
lz = len(zs)

def get_cube(x,y,z):
    a_x, a_y, a_z = len(xs[xs <= x]) - 1, len(ys[ys <= y]) - 1, len(zs[zs <= z]) - 1
    corner_a = (ly * lz) * a_x + (lz) * a_y + a_z
    corner_b = corner_a + lz
    corner_c = corner_a + ly * lz
    corner_d = corner_a + ly * lz + lz
    index_list = [corner_a,corner_a+1,corner_b,corner_b+1,
                  corner_c,corner_c+1,corner_d,corner_d+1]
    return df.iloc[index_list]

In [163]:
def ds_interp(x,y,z,method="linear"):
    cube = get_cube(x,y,z)
    xs = cube.X.values
    ys = cube.Y.values
    zs = cube.Z.values
    bx = griddata((xs,ys,zs),test_cube.Bx.values, (x,y,z), method=method).item()
    by = griddata((xs,ys,zs),test_cube.By.values, (x,y,z), method=method).item()
    bz = griddata((xs,ys,zs),test_cube.Bz.values, (x,y,z), method=method).item()
    return bx,by,bz

In [136]:
ds_interp(.3,.7,6.)

(609.1172983136797, 1393.0642843639996, 14657.852810748002)

In [137]:
%timeit ds_interp(.3,.7,6.)

1.13 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [138]:
ds_interp(.31,.71,6.)

(651.3194290775997, 1464.1326597159991, 14682.942385180002)

In [140]:
%timeit ds_interp(.31,.71,6.)

1.1 ms ± 6.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- So, this is the best timing so far for getting a magnetic field at an arbitrary point. Also it does not really on a potentially questionable field map.
- **Idea for Offline once field is measured:** Model measured points on a dense grid (25 mm, 10mm, 1mm?? Can be a long job...it only needs to run one time! Will likely depend on what doesn't slow down interpolation), then use dense grid as input for interpolation.
- **Brian's Idea:** Write a nice speedy field calculation function, use that for Geant (is this possible?)
    - Will definitely need to discuss this with Offline experts to see implementation and if it's possible / worthwhile.

In [134]:
test_cube

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
3071933,0.300,0.700,5.996,606.750687,1387.768884,14689.461393,0.761577,1.165905,-11.022933,1514.571678
3071934,0.300,0.700,6.021,621.542007,1420.865136,14491.907756,0.761577,1.165905,-11.581049,1550.818520
3072454,0.300,0.725,5.996,644.054087,1524.945714,14741.128184,0.784618,1.178456,-12.051089,1655.330743
3072455,0.300,0.725,6.021,662.761117,1568.378528,14515.159227,0.784618,1.178456,-12.730086,1702.616060
3122470,0.325,0.700,5.996,673.386331,1424.128368,14711.884665,0.771767,1.136126,-11.050883,1575.267798
3122471,0.325,0.700,6.021,691.174252,1461.007069,14502.099405,0.771767,1.136126,-11.654649,1616.207806
3122991,0.325,0.725,5.996,716.389323,1568.364912,14768.911133,0.794512,1.149377,-12.163010,1724.190889
3122992,0.325,0.725,6.021,738.785185,1616.527557,14528.472062,0.794512,1.149377,-12.898228,1777.300911


In [142]:
%load_ext line_profiler

In [143]:
%lprun -f ds_interp ds_interp(.31,.71,6.)

In [164]:
%lprun -f get_cube get_cube(.31,.71,6.)

## Interpolator Quick #1

In [172]:
xs = df.X.unique()
ys = df.Y.unique()
zs = df.Z.unique()

dx = xs[1]-xs[0]
dy = ys[1]-ys[0]
dz = zs[1]-zs[0]

lx = len(xs)
ly = len(ys)
lz = len(zs)

df_np = df[["X","Y","Z","Bx","By","Bz"]].values

def get_cube(x,y,z):
    a_x, a_y, a_z = len(xs[xs <= x]) - 1, len(ys[ys <= y]) - 1, len(zs[zs <= z]) - 1
    corner_a = (ly * lz) * a_x + (lz) * a_y + a_z
    corner_b = corner_a + lz
    corner_c = corner_a + ly * lz
    corner_d = corner_a + ly * lz + lz
    index_list = [corner_a,corner_a+1,corner_b,corner_b+1,
                  corner_c,corner_c+1,corner_d,corner_d+1]
    return df_np[index_list]

def ds_interp(x,y,z,method="linear"):
    cube = get_cube(x,y,z)
    bx = griddata((cube[:,0],cube[:,1],cube[:,2]),cube[:,3], (x,y,z), method=method).item()
    by = griddata((cube[:,0],cube[:,1],cube[:,2]),cube[:,4], (x,y,z), method=method).item()
    bz = griddata((cube[:,0],cube[:,1],cube[:,2]),cube[:,5], (x,y,z), method=method).item()
    return bx,by,bz

In [181]:
%lprun -f ds_interp ds_interp(.31,.71,6.)

In [183]:
%timeit ds_interp(.31,.71,6.)

733 µs ± 5.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- **WORKING, and fast:** Now the bottleneck is the call to griddata. Try scipy.interpolate.RegularGridInterpolator
- Current Speed Improvement: **50x faster than past interpolator**, 3x faster than model eval

## Test of hard code
- replaced old "mapinterp.py" code with this newer, faster function
- testing it here!

In [225]:
from mu2e.tools.mapinterp import get_df_interp_func

In [ ]:
#mau13_nearest = get_ds_interp_func(df,method='nearest')
#mau13_interp = get_ds_interp_func(df,method='linear')

In [226]:
mau13_nearest_fast = get_df_interp_func(df,method='nearest')
mau13_interp_fast = get_df_interp_func(df,method='linear')

In [190]:
%timeit mau13_interp(0.156,-.699,7.001)

33.4 ms ± 2.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [191]:
%timeit mau13_nearest(0.156,-.699,7.001)

31.2 ms ± 950 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [192]:
%timeit mau13_interp_fast(0.156,-.699,7.001)

737 µs ± 5.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [227]:
ds_interp(0.156,-.699,7.001)

TypeError: tuple indices must be integers or slices, not tuple

In [228]:
mau13_interp_fast(0.156,-.699,7.001)

(94.57883452204001, -446.1719677802401, 11347.763332736002)

In [193]:
%timeit mau13_nearest_fast(0.156,-.699,7.001)

275 µs ± 6.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [195]:
mau13_nearest_fast(0.156,-.699,7.001)

(96.407603304, -472.752239114, 11351.8362832)

It works!

## Trilinear Interpolation Hand Code
- See if this is quicker than the less specialized functions in scipy.interpolate.
- First switching to testing trajectory code again.

In [230]:
test_cube

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
3071933,0.300,0.700,5.996,606.750687,1387.768884,14689.461393,0.761577,1.165905,-11.022933,1514.571678
3071934,0.300,0.700,6.021,621.542007,1420.865136,14491.907756,0.761577,1.165905,-11.581049,1550.818520
3072454,0.300,0.725,5.996,644.054087,1524.945714,14741.128184,0.784618,1.178456,-12.051089,1655.330743
3072455,0.300,0.725,6.021,662.761117,1568.378528,14515.159227,0.784618,1.178456,-12.730086,1702.616060
3122470,0.325,0.700,5.996,673.386331,1424.128368,14711.884665,0.771767,1.136126,-11.050883,1575.267798
3122471,0.325,0.700,6.021,691.174252,1461.007069,14502.099405,0.771767,1.136126,-11.654649,1616.207806
3122991,0.325,0.725,5.996,716.389323,1568.364912,14768.911133,0.794512,1.149377,-12.163010,1724.190889
3122992,0.325,0.725,6.021,738.785185,1616.527557,14528.472062,0.794512,1.149377,-12.898228,1777.300911


In [236]:
xs = df.X.unique()
ys = df.Y.unique()
zs = df.Z.unique()

dx = xs[1]-xs[0]
dy = ys[1]-ys[0]
dz = zs[1]-zs[0]

lx = len(xs)
ly = len(ys)
lz = len(zs)

df_np = df[["X","Y","Z","Bx","By","Bz"]].values

def get_cube(x,y,z):
    a_x, a_y, a_z = len(xs[xs <= x]) - 1, len(ys[ys <= y]) - 1, len(zs[zs <= z]) - 1
    corner_a = (ly * lz) * a_x + (lz) * a_y + a_z
    corner_b = corner_a + lz
    corner_c = corner_a + ly * lz
    corner_d = corner_a + ly * lz + lz
    index_list = [corner_a,corner_a+1,corner_b,corner_b+1,
                  corner_c,corner_c+1,corner_d,corner_d+1]
    return df_np[index_list]

def ck_interp(xd,yd,zd,ff):
    c00 = ff[0,0,0]*(1 - xd) + ff[1,0,0] * xd 
    c01 = ff[0,0,1]*(1 - xd) + ff[1,0,1] * xd 
    c10 = ff[0,1,0]*(1 - xd) + ff[1,1,0] * xd 
    c11 = ff[0,1,1]*(1 - xd) + ff[1,1,1] * xd 
    
    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd
    return c0 * (1 - zd) + c1 * zd

def ck_interp2(x,y,z,f):
    a0 = (f[0,0,0]*x[1]*y[1]*z[1]/((x[0]-x[1])*(y[0]-y[1])*(z[1]-z[0])) +
          f[0,0,0]*x[1]*y[1]*z[1]/((x[0]-x[1])*(y[0]-y[1])*(z[1]-z[0])) +
          f[0,0,0]*x[1]*y[1]*z[1]/((x[0]-x[1])*(y[0]-y[1])*(z[1]-z[0])) +
          f[0,0,0]*x[1]*y[1]*z[1]/((x[0]-x[1])*(y[0]-y[1])*(z[1]-z[0])) +
          f[0,0,0]*x[1]*y[1]*z[1]/((x[0]-x[1])*(y[0]-y[1])*(z[1]-z[0])) +
          f[0,0,0]*x[1]*y[1]*z[1]/((x[0]-x[1])*(y[0]-y[1])*(z[1]-z[0]))
    a1 = 
    a2 = 
    a3 = 
    a4 = 
    a5 =
    a6 = 
    a7 = 
    return a0 + a1 * x + a2 * y + a3 * z + a4 * x * y + a5 * x * z + a6 * y * z + a7 * x * y * z

def ds_interp_ck(x,y,z,method=1):
    cube = get_cube(x,y,z)
    
    xx = np.unique(cube[:,0])
    yy = np.unique(cube[:,1])
    zz = np.unique(cube[:,2])
    
    #xx = cube[:,0].reshape((2,2,2))
    #yy = cube[:,1].reshape((2,2,2))
    #zz = cube[:,2].reshape((2,2,2))
    bxs = cube[:,3].reshape((2,2,2))
    bys = cube[:,4].reshape((2,2,2))
    bzs = cube[:,5].reshape((2,2,2))
    
    if method == 1:
        xd = (x-xx[0])/(xx[1]-xx[0])
        yd = (y-yy[0])/(yy[1]-yy[0])
        zd = (z-zz[0])/(zz[1]-zz[0])

        bx = ck_interp(xd,yd,zd, bxs)
        by = ck_interp(xd,yd,zd, bys)
        bz = ck_interp(xd,yd,zd, bzs)
    elif method == 2:
        bx = ck_interp2(xx,yy,zz, bxs)
        by = ck_interp2(xx,yy,zz, bys)
        bz = ck_interp2(xx,yy,zz, bzs)
    
    return bx,by,bz

In [237]:
ds_interp_ck(0.156,-.699,7.001)

(94.20486739607203, -445.37895963504906, 11348.012744296258)

In [241]:
mau13_interp_fast(0.156,-.699,7.001)

(94.57883452204001, -446.1719677802401, 11347.763332736002)

In [242]:
%timeit ds_interp_ck(0.156,-.699,7.001)

46.8 µs ± 764 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [244]:
%timeit mau13_interp_fast(0.156,-.699,7.001)

733 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Wow! Another x15 improvement! It's interesting that griddata is so much slower. Note that the values do not exactly agree, but are very close.

In [246]:
38500 / 46.8

822.6495726495727

**Overall speedup of x822 faster than original interp function**

## RegularGridInterpolator Test
- need to restructure Bx,By,Bz as 3d grids with 'ijk' indexing. i.e. bx[x,y,z]

In [187]:
from scipy.interpolate import RegularGridInterpolator

In [202]:
X = test_cube.X
Bx = test_cube.Bx

In [205]:
Bxx = Bx.values.reshape((2,2,2))

In [204]:
X.values.reshape((2,2,2))

array([[[0.3  , 0.3  ],
        [0.3  , 0.3  ]],

       [[0.325, 0.325],
        [0.325, 0.325]]])

In [203]:
X,Bx

(3071933    0.300
 3071934    0.300
 3072454    0.300
 3072455    0.300
 3122470    0.325
 3122471    0.325
 3122991    0.325
 3122992    0.325
 Name: X, dtype: float64, 3071933    606.750687
 3071934    621.542007
 3072454    644.054087
 3072455    662.761117
 3122470    673.386331
 3122471    691.174252
 3122991    716.389323
 3122992    738.785185
 Name: Bx, dtype: float64)

In [206]:
Bxx

array([[[606.75068713, 621.54200703],
        [644.05408653, 662.76111671]],

       [[673.38633052, 691.17425191],
        [716.3893231 , 738.78518477]]])

In [208]:
Bxx[0,1,1]

662.761116712

In [209]:
test_cube

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
3071933,0.300,0.700,5.996,606.750687,1387.768884,14689.461393,0.761577,1.165905,-11.022933,1514.571678
3071934,0.300,0.700,6.021,621.542007,1420.865136,14491.907756,0.761577,1.165905,-11.581049,1550.818520
3072454,0.300,0.725,5.996,644.054087,1524.945714,14741.128184,0.784618,1.178456,-12.051089,1655.330743
3072455,0.300,0.725,6.021,662.761117,1568.378528,14515.159227,0.784618,1.178456,-12.730086,1702.616060
3122470,0.325,0.700,5.996,673.386331,1424.128368,14711.884665,0.771767,1.136126,-11.050883,1575.267798
3122471,0.325,0.700,6.021,691.174252,1461.007069,14502.099405,0.771767,1.136126,-11.654649,1616.207806
3122991,0.325,0.725,5.996,716.389323,1568.364912,14768.911133,0.794512,1.149377,-12.163010,1724.190889
3122992,0.325,0.725,6.021,738.785185,1616.527557,14528.472062,0.794512,1.149377,-12.898228,1777.300911


In [218]:
xs = df.X.unique()
ys = df.Y.unique()
zs = df.Z.unique()

dx = xs[1]-xs[0]
dy = ys[1]-ys[0]
dz = zs[1]-zs[0]

lx = len(xs)
ly = len(ys)
lz = len(zs)

df_np = df[["X","Y","Z","Bx","By","Bz"]].values

def get_cube(x,y,z):
    a_x, a_y, a_z = len(xs[xs <= x]) - 1, len(ys[ys <= y]) - 1, len(zs[zs <= z]) - 1
    corner_a = (ly * lz) * a_x + (lz) * a_y + a_z
    corner_b = corner_a + lz
    corner_c = corner_a + ly * lz
    corner_d = corner_a + ly * lz + lz
    index_list = [corner_a,corner_a+1,corner_b,corner_b+1,
                  corner_c,corner_c+1,corner_d,corner_d+1]
    return df_np[index_list], a_x, a_y, a_z

def ds_interp_reg(x,y,z,method="linear"):
    cube, a_x, a_y, a_z = get_cube(x,y,z)
    bxs = cube[:,3].reshape((2,2,2))
    bys = cube[:,4].reshape((2,2,2))
    bzs = cube[:,5].reshape((2,2,2))
    
    #print(f"Bxs = {bxs}")
    #print(f"Bys = {bys}")
    #print(f"Bzs = {bzs}")
    
    xss = np.array([xs[a_x],xs[a_x+1]])
    yss = np.array([ys[a_y],ys[a_y+1]])
    zss = np.array([zs[a_z],zs[a_z+1]])
    
    bx_interp = RegularGridInterpolator((xss,yss,zss), bxs)
    by_interp = RegularGridInterpolator((xss,yss,zss), bys)
    bz_interp = RegularGridInterpolator((xss,yss,zss), bzs)
    
    bx = bx_interp([x,y,z]).item()
    by = by_interp([x,y,z]).item()
    bz = bz_interp([x,y,z]).item()
    
    #bx = griddata((cube[:,0],cube[:,1],cube[:,2]),cube[:,3], (x,y,z), method=method).item()
    #by = griddata((cube[:,0],cube[:,1],cube[:,2]),cube[:,4], (x,y,z), method=method).item()
    #bz = griddata((cube[:,0],cube[:,1],cube[:,2]),cube[:,5], (x,y,z), method=method).item()
    return bx,by,bz

In [219]:
ds_interp_reg(0,0,6)

(4.347280929439999, -5.188215101999999, 14532.959472028)

In [223]:
mau13_interp_fast(0.156,-.699,7.001)

(94.57883452204001, -446.1719677802401, 11347.763332736002)

In [229]:
mau13_interp_fast(0.156,-.699,7.001,True)

(94.57883452204001, -446.1719677802401, 11347.763332736002)

In [221]:
ds_interp_reg(0.156,-.699,7.001)

(94.20486739607202, -445.378959635049, 11348.012744296257)

In [222]:
%timeit ds_interp_reg(0.156,-.699,7.001)

767 µs ± 28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- Results vary just slightly, and this method is not any quicker.